In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
# import seaborn as sns
# sns.set()
# sns.set_style("white")
# sns.set_palette("GnBu_d")

# also import these "new" libraries 
# Note: you may have to download an add them to your environment (using e.g. 'conda install -c conda-forge folium')

import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects
import math
import random
from pathlib import Path

import geopy
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic

In [2]:
df = pd.read_csv("../all_station_id.csv")

In [3]:
df2 = pd.read_csv("../boston_2016_distance_duration_speed.csv")

In [4]:
df2.drop(["Unnamed: 0"], axis=1, inplace=True)

In [5]:
# Drop Trips with the 8D OPS stations

df2.drop([598580], axis=0, inplace=True)
df2.drop([598729], axis=0, inplace=True)
df2.drop([1072609], axis=0, inplace=True)
#########################################
# Löschen alle die schneller als 30km/h sind

df2.drop([88647], axis=0, inplace=True)
df2.drop([88649], axis=0, inplace=True)
df2.drop([17997], axis=0, inplace=True)
df2.drop([89416], axis=0, inplace=True)
df2.drop([824312], axis=0, inplace=True)
df2.drop([1143845], axis=0, inplace=True)
#########################################
# Löschen alle die speed<0 haben

df2.drop([1114233], axis=0, inplace=True)
df2.drop([1114235], axis=0, inplace=True)
df2.drop([1114234], axis=0, inplace=True)

In [8]:
# Löscht alle Roundtrips
df2 = df2.drop(df2[df2['Distance (KM)'] == 0].index)
# Löscht alle Trips die länger als 8Std sind
df2 = df2.drop(df2[df2['Duration (Seconds)'] > 28800].index)

In [9]:
dingo = (df2.where(lambda x: (x.start_station_id == 8.0), axis = 0))
len(df2) - dingo['start_station_id'].isnull().sum()

4022

In [10]:
DasDingo = []

In [11]:
for i in df.start_station_id:
    dingo = (df2.where(lambda x: (x.start_station_id == i), axis = 0))
    DasDingo.append(len(df2) - dingo['start_station_id'].isnull().sum())

In [12]:
print(DasDingo)

[64, 4542, 8783, 6659, 11399, 4408, 4022, 9270, 9202, 8420, 7799, 7679, 8639, 2516, 14702, 6212, 6953, 10498, 11692, 31026, 14062, 7484, 8243, 7824, 4930, 3627, 6209, 5239, 9024, 13467, 9282, 16033, 1233, 8902, 9039, 10495, 8390, 13396, 10810, 7539, 9788, 12697, 13587, 7924, 7461, 8429, 2699, 8903, 16466, 11120, 11733, 1444, 749, 12923, 7811, 15198, 12514, 3170, 9451, 2900, 4872, 34496, 23804, 6087, 8489, 5713, 10424, 11371, 22376, 11308, 14026, 2129, 5443, 6367, 20694, 10138, 2840, 8939, 5212, 10029, 12933, 7693, 11802, 14367, 962, 1373, 5398, 11213, 9784, 9154, 10830, 3486, 10025, 2172, 3132, 6255, 11980, 486, 14076, 8406, 12610, 9242, 1653, 1338, 2382, 3147, 10092, 9517, 7038, 5228, 3805, 8419, 4683, 3538, 2164, 2576, 990, 1017, 2405, 701, 3544, 2667, 5847, 3241, 4212, 1567, 920, 5987, 2561, 6479, 2587, 5948, 1836, 2194, 6010, 4902, 7373, 6371, 0, 0, 1705, 4319, 4938, 269, 4634, 1003, 2823, 300, 2349, 442, 895, 1915, 3246, 11551, 14705, 17130, 2291, 314, 774, 551, 4530, 17663, 8747,

In [13]:
#df['amout_of_trips_per_station'] = DasDingo(lambda x: (x.DasDingo), axis = 0)
df['amout_of_trips_per_station'] = pd.DataFrame (DasDingo, columns = ['amout_of_trips_per_station'])

In [14]:
df

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station
0,4748,1,18 Dorrance Warehouse,42.387151,-71.075978,64
1,26681,3,Colleges of the Fenway,42.340021,-71.100812,4542
2,97049,4,Tremont St. at Berkeley St.,42.345392,-71.069616,8783
3,161457,5,Northeastern U / North Parking Lot,42.341814,-71.090179,6659
4,103358,6,Cambridge St. at Joy St.,42.361174,-71.065142,11399
...,...,...,...,...,...,...
184,29748,215,The Eddy at New Street,42.370785,-71.044262,36
185,72713,216,Glendon St at Condor St,42.382447,-71.030232,23
186,71084,217,Orient Heights T Stop - Bennington St at Sarat...,42.386732,-71.006212,43
187,30227,218,Watermark Seaport - Boston Wharf Rd at Seaport...,42.351586,-71.045693,1364


In [15]:
# filepath = Path('/Users/daniel/Desktop/boston_2016_distance_duration_speed.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True)
# df.to_csv(filepath)

In [16]:
sum(df['amout_of_trips_per_station'])

1195289

In [17]:
df.sort_values(['amout_of_trips_per_station'],axis=0, ascending=False,inplace=True,na_position='first')

In [18]:
df.head()

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station
61,97694,67,MIT at Mass Ave / Amherst St,42.358100,-71.093198,34496
19,57721,22,South Station - 700 Atlantic Ave.,42.352175,-71.055547,31026
62,41216,68,Central Square at Mass Ave / Essex St,42.365070,-71.103100,23804
68,20008,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,22376
74,6903,80,MIT Stata Center at Vassar St / Main St,42.361962,-71.092053,20694


In [31]:
df["start_coordinates"] = list(zip(df["start_station_latitude"],df["start_station_longitude"]))

In [27]:
center_point = (42.36041042227672,-71.05799197093351)
type(center_point)

tuple

In [32]:
def calc_distances_from_center(start_coordinates):
    return geopy.distance.geodesic( start_coordinates,center_point).km

In [33]:
df['distance_center_(km)'] = df.apply(lambda row: calc_distances_from_center(row.start_coordinates), axis=1)

In [40]:
df.sort_values(['distance_center_(km)'],axis=0, ascending=False,inplace=True,na_position='first')
df.head(5)

,Unnamed: 0,start_station_id,start_station_name,start_station_latitude,start_station_longitude,amout_of_trips_per_station,start_coordinates,distance_center_(km)
177,26226,208,Oak Square - 615 Washington St,42.350570,-71.166491,185,"(42.35057, -71.166491)",9.005604
150,95299,174,Washington St at Brock St,42.349005,-71.160411,895,"(42.34900494405213, -71.1604106426239)",8.532756
157,7336,183,Alewife MBTA at Steel Place,42.395588,-71.142606,314,"(42.39558845716709, -71.14260613918304)",7.989504
130,116577,142,Alewife Station at Russell Field,42.396105,-71.139459,2587,"(42.396105, -71.139459)",7.793473
151,124454,175,Brighton Center,42.348949,-71.150272,1915,"(42.34894857235593, -71.15027189254761)",7.708726
